# Generate an ndarray on a regular grid of the NAM 2017 model

This notebook and the programs/code used within will create a 4D numpy array of the mode for a specified dz and depth.  A 3D array for each of the following properties: VP, VS, RHO, and Qu.  The piece-wise model had an x and y resolution of $dx=dy=50m$. Therefore, to refine the mesh at a courser grid, the user must interpolate after the numpy array is created.

The Vo_maps.txt and horizons.txt file were provided to me from KNMI.  These file discribe the NAM 2017 model.  Please see that report for more information.  I have edited these .txt file to remove the first line which made it easy to use pandas to "automatically" read these files.

NOTE: The gnam python codes needs to be in this current directory. Hopefully to be made into a package at a later date.


### Step 1

Show contents of the 'nam_model_data_in' directory. There should be two files: Vo_maps.txt and horizons.txt

In [ ]:
!ls 
#!ls nam_model_data_in

### Step 2

Use the nammodel code to generate the array.

In [ ]:
from gnam.model.nammodel import nammodel as nam

data_in_dir  = './nam_model_data_in/'
fqn_vo_map   = data_in_dir + 'Vo_maps.txt'
fqn_horizons = data_in_dir + 'horizons.txt'


# Obstantiate the NAM model object
nam_model = nam(fqn_vo_map,fqn_horizons)

# Generate and return the ndarray and the header/meta data on x,y,z
dz = 10
maxDepth = 6000
nam_data = nam_model.computeGriddedModel3D(dz,maxDepth)

# Get just the ndarray
nam_props = nam_data[0] 

print('Model.shape:',nam_props.shape)


### Step 3

Prepare to compress and pickle numpy array: make sure the path and file name are good and varify x,y,x "header" data.

In [ ]:
# fqn
out_dir = './model_data/'
fname   = 'raw_nam_2017_vp_vs_rho_Q_model_dz' + str(dz) + '_depth' + str(maxDepth) + '.npz'
ofqn    = out_dir + fname

print('Output FQN:\n',ofqn)
print()


# header data
import numpy as np
xdata = nam_data[1]
ydata = nam_data[2]
zdata = nam_data[3]

print('Header Data:')
print(' x-header:',xdata.astype(np.int32))
print(' y-header:',ydata.astype(np.int32))
print(' z-header:',zdata.astype(np.int32))

### Step 4

Compress and store the numpy array of the NAM model

In [ ]:
import numpy as np

xdata = nam_data[1]
ydata = nam_data[2]
zdata = nam_data[3]

np.savez_compressed(ofqn,props=nam_props,xd=xdata,yd=ydata,zd=zdata)
print(ofqn)

### Step 5

Deompress ndarray of the NAM model as a QC

In [2]:
import numpy as np

# copy from the line above, or put in the Fully Qualified Name of the file
ifqn = './model_data/raw_nam_2017_vp_vs_rho_Q_model_dz10_depth6000.npz'

#decompress
data = np.load(ifqn)
props = data['props'] #4D ndarray of subsurface model


#header/meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('Header Data:')
print(' x:',xdata)
print(' y:',ydata)
print(' z:',zdata)
print()


# individual parameters from the headers 
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx #notice that this can be computed

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy #notice that this can be computed

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz #notice that this can be computed


print('Individual Header Data Parameters:')
print(' xmin,dx,nx,xmax = %d,%d,%d,%d' %(xmin,dx,nx,xmax))
print(' ymin,dy,ny,ymax = %d,%d,%d,%d' %(ymin,dy,ny,ymax))
print(' zmin,dz,nz,zmax = %d,%d,%d,%d' %(zmin,dz,nz,zmax))


Header Data:
 x: [2.074625e+05 5.000000e+01 1.450000e+03]
 y: [5.559625e+05 5.000000e+01 1.198000e+03]
 z: [  0.  10. 601.]

Individual Header Data Parameters:
 xmin,dx,nx,xmax = 207462,50,1450,279912
 ymin,dy,ny,ymax = 555962,50,1198,615812
 zmin,dz,nz,zmax = 0,10,601,6000


### Step 6

Plot a depth slice. NOTE: chose which property you want to see: 0=vp, 1=vs, 2=rho, 3=Q

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

depth = 100 # This is an INDEX! So, know your header data and its mapping to the model!
iprop = 0

# index order: p      x  y  z
surf   = props[iprop, :, :, depth].transpose()
print('surf.shape',surf.shape)

# Get min max to normalize surface
vp_min = np.min(surf[1])
vp_max = np.max(surf[1])
print('vp_min:',vp_min)
print('vp_max:',vp_max)
surf_norm = Normalize(vp_min,vp_max)

# compute coordinates
xc  = dx*np.arange(nx) + xmin
yc  = dy*np.arange(ny) + ymin
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()


### Step 7

See the "Using_GriddedModel3D.ipynb" notebook.  This notebook shows how the model can be manipulated or processed such as smoothing it and slicing it.